In [41]:
import re
import json
import hashlib
from pathlib import Path
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Any, Set, Tuple
from enum import Enum
from collections import defaultdict

In [42]:
class NodeType(Enum):
    """Types of nodes in the document hierarchy"""
    DOCUMENT = "document"
    CHAPTER = "chapter"
    SECTION = "section"
    SUBSECTION = "subsection"
    SUBSUBSECTION = "subsubsection"
    PARAGRAPH = "paragraph"
    SUBPARAGRAPH = "subparagraph"
    # Leaf nodes
    SENTENCE = "sentence"
    BLOCK_FORMULA = "block_formula"
    FIGURE = "figure"
    TABLE = "table"
    # Special
    ABSTRACT = "abstract"
    ACKNOWLEDGMENTS = "acknowledgments"
    APPENDIX = "appendix"


# Hierarchy order for determining nesting levels
HIERARCHY_ORDER = [
    NodeType.DOCUMENT,
    NodeType.CHAPTER,
    NodeType.SECTION,
    NodeType.SUBSECTION,
    NodeType.SUBSUBSECTION,
    NodeType.PARAGRAPH,
    NodeType.SUBPARAGRAPH,
]

# Leaf node types (smallest elements)
LEAF_TYPES = {NodeType.SENTENCE, NodeType.BLOCK_FORMULA, NodeType.FIGURE, NodeType.TABLE}


@dataclass
class HierarchyNode:
    """A node in the document hierarchy tree"""
    node_type: NodeType
    title: str = ""
    content: str = ""
    children: List['HierarchyNode'] = field(default_factory=list)
    label: str = ""
    source_file: str = ""
    content_hash: str = ""
    unique_id: str = ""
    
    def __post_init__(self):
        if not self.unique_id:
            self.unique_id = hashlib.md5(
                f"{self.node_type.value}:{self.title}:{self.content[:100]}".encode()
            ).hexdigest()[:12]
        if not self.content_hash and self.content:
            self.content_hash = hashlib.md5(self.content.encode()).hexdigest()
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert node to dictionary for JSON serialization"""
        return {
            "type": self.node_type.value,
            "title": self.title,
            "content": self.content if self.node_type in LEAF_TYPES else "",
            "label": self.label,
            "source_file": self.source_file,
            "content_hash": self.content_hash,
            "unique_id": self.unique_id,
            "children": [child.to_dict() for child in self.children]
        }


@dataclass 
class BibEntry:
    """A bibliography entry"""
    key: str
    entry_type: str
    fields: Dict[str, str] = field(default_factory=dict)
    
    def to_bibtex(self) -> str:
        """Convert to BibTeX format"""
        lines = [f"@{self.entry_type}{{{self.key},"]
        for key, value in self.fields.items():
            lines.append(f"  {key} = {{{value}}},")
        lines.append("}")
        return "\n".join(lines)
    
    def content_hash(self) -> str:
        """Generate hash based on content for deduplication"""
        # Normalize fields for comparison
        normalized = {k.lower(): v.lower().strip() for k, v in self.fields.items()}
        content = json.dumps(normalized, sort_keys=True)
        return hashlib.md5(content.encode()).hexdigest()


In [43]:
class LaTeXFileGatherer:
    # Patterns to match \input and \include commands
    PATTERN = re.compile(r'(?m)^(?![^%\n]*%).*\\(?:input|include)\{([^}]+)\}')
    
    def __init__(self, base_dir: str):
        self.base_dir = Path(base_dir)
        self.included_files: Set[str] = set()
        self.file_contents: Dict[str, str] = {}
        self.file_order: List[str] = []
        
    def gather_files(self, main_file: str) -> Dict[str, str]:
        """
        Recursively gather all files starting from the main file.
        Returns a dict mapping file paths to their contents.
        """
        self.included_files.clear()
        self.file_contents.clear()
        self.file_order.clear()
        
        main_path = self.base_dir / main_file
        self._process_file(main_path)
        
        return self.file_contents
    
    def _resolve_path(self, include_path: str, current_file: Path) -> Path:
        """Resolve the path of an included file"""
        # Add .tex extension if not present
        if not include_path.endswith('.tex'):
            include_path += '.tex'
        
        # Try relative to current file first
        relative_path = current_file.parent / include_path
        if relative_path.exists():
            return relative_path
        
        # Try relative to base directory
        base_relative = self.base_dir / include_path
        if base_relative.exists():
            return base_relative
        
        return relative_path  # Return even if doesn't exist for error reporting
    
    def _process_file(self, file_path: Path) -> str:
        """Process a single file and recursively process includes"""
        # Normalize path for tracking
        normalized_path = str(file_path.resolve())
        
        if normalized_path in self.included_files:
            return ""  # Already processed
        
        if not file_path.exists():
            print(f"Warning: File not found: {file_path}")
            return ""
        
        self.included_files.add(normalized_path)
        self.file_order.append(normalized_path)
        
        try:
            with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                content = f.read()
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return ""
        
        # Store original content
        self.file_contents[normalized_path] = content
        
        # Find and process all includes
        expanded_content = self._expand_includes(content, file_path)
        
        return expanded_content
    
    def _expand_includes(self, content: str, current_file: Path) -> str:
        """Expand \input and \include commands in content"""
        result = content
        
        # Process \input commands
        for match in self.PATTERN.finditer(content):
            include_path = match.group(1)
            resolved_path = self._resolve_path(include_path, current_file)
            included_content = self._process_file(resolved_path)
        
        # Process \include commands  
        for match in self.PATTERN.finditer(content):
            include_path = match.group(1)
            resolved_path = self._resolve_path(include_path, current_file)
            included_content = self._process_file(resolved_path)
        
        return result
    
    def get_merged_content(self) -> str:
        """Get all content merged into a single string with file markers"""
        merged = []
        for file_path in self.file_order:
            content = self.file_contents.get(file_path, "")
            # Add file marker for tracking
            merged.append(f"%%% FILE: {file_path} %%%\n")
            merged.append(content)
            merged.append("\n")
        return "\n".join(merged)
    
    def get_unused_files(self) -> Set[str]:
        """Get files that exist but are not part of compilation"""
        all_files = set()
        for tex_file in self.base_dir.rglob("*.tex"):
            all_files.add(str(tex_file.resolve()))
        return all_files - self.included_files


In [44]:
class LaTeXCleaner:
    """Cleans and normalizes LaTeX content"""
    
    # Commands to remove (no semantic meaning)
    REMOVE_COMMANDS = [
        r'\\centering',
        r'\\raggedright',
        r'\\raggedleft',
        r'\\noindent',
        r'\\smallskip',
        r'\\medskip',
        r'\\bigskip',
        r'\\newpage',
        r'\\clearpage',
        r'\\pagebreak',
        r'\\linebreak',
        r'\\hfill',
        r'\\vfill',
        r'\\hspace\{[^}]*\}',
        r'\\vspace\{[^}]*\}',
        r'\\phantom\{[^}]*\}',
        r'\\hphantom\{[^}]*\}',
        r'\\vphantom\{[^}]*\}',

        r'\\par',
        r'\\parindent',
        r'\\parskip',
        r'\\baselineskip',
        r'\\stretch\{[^}]*\}',

        # Font / style
        r'\\textbf\{([^}]*)\}',
        r'\\textit\{([^}]*)\}',
        r'\\emph\{([^}]*)\}',
        r'\\underline\{([^}]*)\}',
        r'\\texttt\{([^}]*)\}',
        r'\\bfseries',
        r'\\itshape',
        r'\\ttfamily',

        r'\\item',
        r'\\item\[[^\]]*\]',

    ]
    
    # Table formatting commands to remove
    TABLE_FORMATTING = [
        r'\\toprule',
        r'\\midrule',
        r'\\bottomrule',
        r'\\hline',
        r'\\cline\{[^}]*\}',
        r'\\multicolumn\{[^}]*\}\{[^}]*\}',
        r'\\multirow\{[^}]*\}\{[^}]*\}',
        
        r'\\addlinespace',
        r'\\cmidrule\{[^}]*\}', 
    ]
    
    # Figure/table placement specifiers
    PLACEMENT_SPECS = re.compile(r'\[([htbp!]+)\]')
    
    # Inline math patterns (to normalize to $...$)
    INLINE_MATH_PATTERNS = [
        (re.compile(r'\\[(](.*?)\\[)]', re.DOTALL), r'$\1$'),  # \(...\)
        (re.compile(r'\\begin\{math\}(.*?)\\end\{math\}', re.DOTALL), r'$\1$'),
    ]
    
    # Block math environments (to normalize to equation)
    BLOCK_MATH_ENVS = [
        'displaymath', 'eqnarray', 'eqnarray*', 'align', 'align*',
        'gather', 'gather*', 'multline', 'multline*', 'flalign', 'flalign*'
    ]
    
    def __init__(self):
        # Compile removal patterns
        self.remove_patterns = [re.compile(p) for p in self.REMOVE_COMMANDS + self.TABLE_FORMATTING]
    
    def clean(self, content: str) -> str:
        """Apply all cleaning operations"""
        result = content
        
        # Remove comments (but preserve % in math mode)
        result = self._remove_comments(result)
        result = self.normalize_math(result)
        
        # Remove formatting commands
        for pattern in self.remove_patterns:
            result = pattern.sub('', result)
        
        # Remove placement specifiers
        result = self.PLACEMENT_SPECS.sub('', result)
        
        # Normalize whitespace
        result = self._normalize_whitespace(result)
        
        return result
    
    def normalize_math(self, content: str) -> str:
        """Normalize all math expressions"""
        result = content
        
        # Normalize inline math to $...$
        for pattern, replacement in self.INLINE_MATH_PATTERNS:
            result = pattern.sub(replacement, result)
        
        # Normalize $$...$$ to equation environment
        result = re.sub(
            r'\$\$(.*?)\$\$',
            r'\\begin{equation}\1\\end{equation}',
            result,
            flags=re.DOTALL
        )
        
        # Normalize other block math environments to equation
        for env in self.BLOCK_MATH_ENVS:
            pattern = re.compile(
                rf'\\begin\{{{env}\}}(.*?)\\end\{{{env}\}}',
                re.DOTALL
            )
            result = pattern.sub(r'\\begin{equation}\1\\end{equation}', result)

        # result = re.sub(r'~\\mathrm\{([^}]*)\}', r'\1', result)
        # result = re.sub(r'~', '', result)

        return result
    
    def _remove_comments(self, content: str) -> str:
        """Remove LaTeX comments while preserving escaped %"""
        lines = content.split('\n')
        result_lines = []
        for line in lines:
            # Find % that are not escaped
            new_line = []
            i = 0
            while i < len(line):
                if line[i] == '%' and (i == 0 or line[i-1] != '\\'):
                    break  # Rest of line is comment
                new_line.append(line[i])
                i += 1
            result_lines.append(''.join(new_line))
        return '\n'.join(result_lines)
    
    def _normalize_whitespace(self, content: str) -> str:
        """Normalize whitespace in content"""
        # Replace multiple spaces with single space
        result = re.sub(r'[ \t]+', ' ', content)
        # Replace multiple newlines with double newline
        result = re.sub(r'\n{3,}', '\n\n', result)
        return result.strip()
    
    def extract_text_content(self, content: str) -> str:
        """Extract plain text from LaTeX, removing commands"""
        result = content
        
        # Remove \command{...} but keep the content in braces
        result = re.sub(r'\\(?:textbf|textit|textrm|texttt|emph|underline)\{([^}]*)\}', r'\1', result)
        
        # Remove \command without braces
        result = re.sub(r'\\(?:bf|it|rm|tt|em|sc)\b', '', result)
        
        # Remove citations and references (keep for now, mark them)
        result = re.sub(r'\\cite[pt]?\{[^}]*\}', '[CITATION]', result)
        result = re.sub(r'\\ref\{[^}]*\}', '[REF]', result)
        
        # Remove labels
        result = re.sub(r'\\label\{[^}]*\}', '', result)
        
        # Remove remaining simple commands
        result = re.sub(r'\\[a-zA-Z]+\*?(?:\[[^\]]*\])?(?:\{[^}]*\})?', '', result)
        
        # Clean up braces
        result = re.sub(r'[{}]', '', result)
        
        return result.strip()


In [45]:
class HierarchyParser:
    """Parses LaTeX content into a hierarchical structure"""
    
    # Section command patterns with their hierarchy levels
    SECTION_PATTERNS = [
        (r'\\chapter\*?\{([^}]*)\}', NodeType.CHAPTER),
        (r'\\section\*?\{([^}]*)\}', NodeType.SECTION),
        (r'\\subsection\*?\{([^}]*)\}', NodeType.SUBSECTION),
        (r'\\subsubsection\*?\{([^}]*)\}', NodeType.SUBSUBSECTION),
        (r'\\paragraph\*?\{([^}]*)\}', NodeType.PARAGRAPH),
        (r'\\subparagraph\*?\{([^}]*)\}', NodeType.SUBPARAGRAPH),
    ]
    
    # Special sections to detect
    SPECIAL_PATTERNS = [
        (r'\\begin\{abstract\}(.*?)\\end\{abstract\}', NodeType.ABSTRACT),
        (r'\\chapter\*?\{Acknowledgments?\}', NodeType.ACKNOWLEDGMENTS),
        (r'\\section\*?\{Acknowledgments?\}', NodeType.ACKNOWLEDGMENTS),
        (r'\\appendix', NodeType.APPENDIX),
    ]
    
    # Block math environments
    BLOCK_MATH_ENVS = [
        'equation', 'equation*', 'align', 'align*', 'gather', 'gather*',
        'multline', 'multline*', 'eqnarray', 'eqnarray*', 'displaymath'
    ]
    
    # Figure/table environments
    FLOAT_ENVS = ['figure', 'figure*', 'table', 'table*']
    
    # References section patterns (to exclude)
    REFERENCES_PATTERNS = [
        r'\\begin\{thebibliography\}',
        r'\\bibliography\{',
        r'\\printbibliography',
        r'\\section\*?\{References\}',
        r'\\section\*?\{Bibliography\}',
        r'\\chapter\*?\{References\}',
        r'\\chapter\*?\{Bibliography\}',
    ]
    
    def __init__(self, cleaner: LaTeXCleaner = None):
        self.cleaner = cleaner or LaTeXCleaner()
        self._compile_patterns()
        
    def _compile_patterns(self):
        """Compile regex patterns for efficiency"""
        self.section_patterns = [
            (re.compile(pattern, re.DOTALL), node_type)
            for pattern, node_type in self.SECTION_PATTERNS
        ]
        self.special_patterns = [
            (re.compile(pattern, re.DOTALL | re.IGNORECASE), node_type)
            for pattern, node_type in self.SPECIAL_PATTERNS
        ]
        self.references_pattern = re.compile(
            '|'.join(self.REFERENCES_PATTERNS), re.IGNORECASE
        )
        
        # Block math pattern
        math_env_pattern = '|'.join(re.escape(env) for env in self.BLOCK_MATH_ENVS)
        self.block_math_pattern = re.compile(
            rf'\\begin\{{({math_env_pattern})\}}(.*?)\\end\{{\1\}}|'
            rf'\$\$(.*?)\$\$',
            re.DOTALL
        )
        
        # Figure/table pattern
        float_env_pattern = '|'.join(re.escape(env) for env in self.FLOAT_ENVS)
        self.float_pattern = re.compile(
            rf'\\begin\{{({float_env_pattern})\}}(.*?)\\end\{{\1\}}',
            re.DOTALL
        )
        
        # Label pattern
        self.label_pattern = re.compile(r'\\label\{([^}]*)\}')
        
        # Caption pattern
        self.caption_pattern = re.compile(r'\\caption\{([^}]*)\}')
    
    def _is_references_section(self, content: str) -> bool:
        """Check if content is a references/bibliography section"""
        return bool(self.references_pattern.search(content))
    
    def _get_hierarchy_level(self, node_type: NodeType) -> int:
        """Get the hierarchy level of a node type"""
        try:
            return HIERARCHY_ORDER.index(node_type)
        except ValueError:
            return len(HIERARCHY_ORDER)  # Leaf nodes
    
    def _extract_sections(self, content: str, source_file: str = "") -> List[Tuple[int, NodeType, str, str]]:
        """
        Extract all section markers from content.
        Returns list of (position, node_type, title, full_match)
        """
        sections = []
        
        # Find all section commands
        for pattern, node_type in self.section_patterns:
            for match in pattern.finditer(content):
                title = match.group(1).strip()
                sections.append((match.start(), node_type, title, match.group(0)))
        
        # Sort by position
        sections.sort(key=lambda x: x[0])
        return sections
    
    def _extract_block_formulas(self, content: str) -> List[Tuple[int, int, str]]:
        """Extract block formulas with their positions"""
        formulas = []
        for match in self.block_math_pattern.finditer(content):
            formula_content = match.group(2) if match.group(2) else match.group(3)
            formulas.append((match.start(), match.end(), formula_content.strip()))
        return formulas
    
    def _extract_floats(self, content: str) -> List[Tuple[int, int, str, str, str]]:
        """Extract figures and tables with their positions"""
        floats = []
        for match in self.float_pattern.finditer(content):
            env_type = match.group(1)
            env_content = match.group(2)
            
            # Extract label
            label_match = self.label_pattern.search(env_content)
            label = label_match.group(1) if label_match else ""
            
            # Extract caption
            caption_match = self.caption_pattern.search(env_content)
            caption = caption_match.group(1) if caption_match else ""
            
            node_type = NodeType.FIGURE if 'figure' in env_type else NodeType.TABLE
            floats.append((match.start(), match.end(), env_content, label, caption, node_type))
        
        return floats
    
    def _split_into_sentences(self, text: str) -> List[str]:
        """Split text into sentences"""
        # Clean the text first
        cleaned = self.cleaner.extract_text_content(text)
        
        # Split on sentence-ending punctuation
        # Handle common abbreviations
        abbrev_pattern = r'(?<![A-Z])[.!?](?=\s+[A-Z]|\s*$)'
        
        # Simple sentence splitting (handles most cases)
        sentences = re.split(r'(?<=[.!?])\s+(?=[A-Z])', cleaned)
        
        # Filter empty sentences and strip whitespace
        sentences = [s.strip() for s in sentences if s.strip()]
        
        return sentences
    
    def _extract_content_between(self, content: str, start: int, end: int) -> str:
        """Extract content between two positions, excluding formulas and floats"""
        return content[start:end]
    
    def _create_leaf_nodes(self, content: str, source_file: str = "") -> List[HierarchyNode]:
        """Create leaf nodes (sentences, formulas, figures) from content"""
        nodes = []
        
        # Check if this is a references section
        if self._is_references_section(content):
            return nodes
        
        # Extract formulas and floats first
        formulas = self._extract_block_formulas(content)
        floats = self._extract_floats(content)
        
        # Mark positions of formulas and floats
        excluded_ranges = []
        
        for start, end, formula_content in formulas:
            excluded_ranges.append((start, end))
            nodes.append(HierarchyNode(
                node_type=NodeType.BLOCK_FORMULA,
                content=formula_content,
                source_file=source_file
            ))
        
        for start, end, float_content, label, caption, node_type in floats:
            excluded_ranges.append((start, end))
            nodes.append(HierarchyNode(
                node_type=node_type,
                title=caption,
                content=float_content,
                label=label,
                source_file=source_file
            ))
        
        # Sort excluded ranges
        excluded_ranges.sort()
        
        # Extract text between excluded ranges
        text_segments = []
        last_end = 0
        for start, end in excluded_ranges:
            if start > last_end:
                text_segments.append(content[last_end:start])
            last_end = max(last_end, end)
        if last_end < len(content):
            text_segments.append(content[last_end:])
        
        # Split text into sentences
        full_text = ' '.join(text_segments)
        sentences = self._split_into_sentences(full_text)
        
        for sentence in sentences:
            if sentence and len(sentence) > 10:  # Filter very short fragments
                nodes.append(HierarchyNode(
                    node_type=NodeType.SENTENCE,
                    content=sentence,
                    source_file=source_file
                ))
        
        return nodes
    
    def _extract_abstract(self, content: str, source_file: str):
        pattern = re.compile(
            r'\\begin\{abstract\}(.*?)\\end\{abstract\}',
            re.DOTALL | re.IGNORECASE
        )
        match = pattern.search(content)
        if not match:
            return content, None

        abstract_raw = match.group(1)

        abstract_node = HierarchyNode(
            node_type=NodeType.ABSTRACT,
            title="Abstract",
            source_file=source_file
        )

        # 🔥 QUAN TRỌNG: tạo leaf nodes
        abstract_node.children = self._create_leaf_nodes(
            abstract_raw, source_file
        )

        # remove abstract khỏi content chính
        content = content[:match.start()] + content[match.end():]

        return content, abstract_node
    
    def _extract_acknowledgments_content(self, content: str, start: int) -> str:
        """
        Extract acknowledgments content until references/bibliography
        """
        tail = content[start:]

        ref_match = self.references_pattern.search(tail)
        if ref_match:
            return tail[:ref_match.start()]
        return tail


    
    def parse(self, content: str, source_file: str = "") -> HierarchyNode:
        """Parse LaTeX content into a hierarchical tree"""
        # Create root document node
        root = HierarchyNode(
            node_type=NodeType.DOCUMENT,
            title="Document",
            source_file=source_file
        )
        
        # Clean content
        cleaned_content = self.cleaner.clean(content)

        cleaned_content, abstract_node = self._extract_abstract(cleaned_content, source_file)
        if abstract_node:
            root.children.append(abstract_node)
        
        # Extract sections
        sections = self._extract_sections(cleaned_content, source_file)
        
        if not sections:
            # No sections found, create leaf nodes directly under root
            root.children = self._create_leaf_nodes(cleaned_content, source_file)
            return root
        
        # Build hierarchy using a stack
        stack = [(root, -1)]  # (node, hierarchy_level)
        
        for i, (pos, node_type, title, full_match) in enumerate(sections):
            # Check if this is a references section
            is_ack = 'acknowledg' in title.lower()

            if self._is_references_section(title) and not is_ack:
                continue

            
            # Get content until next section
            if i + 1 < len(sections):
                next_pos = sections[i + 1][0]
            else:
                next_pos = len(cleaned_content)
            
            content_start = pos + len(full_match)

            is_ack = 'acknowledg' in title.lower()

            if is_ack:
                section_content = self._extract_acknowledgments_content(
                    cleaned_content, content_start
                )
            else:
                section_content = cleaned_content[content_start:next_pos]
            
            # Check for special sections (acknowledgments, etc.)
            is_acknowledgments = 'acknowledg' in title.lower()
            if is_acknowledgments:
                node_type = NodeType.ACKNOWLEDGMENTS
            
            level = self._get_hierarchy_level(node_type)
            
            # Find parent node
            while stack and stack[-1][1] >= level:
                stack.pop()
            
            parent = stack[-1][0] if stack else root
            
            # Create section node
            section_node = HierarchyNode(
                node_type=NodeType.ACKNOWLEDGMENTS if is_acknowledgments else node_type,
                title=title,
                source_file=source_file
            )

            section_node.children = self._create_leaf_nodes(
                section_content, source_file
            )
            
            # Extract label if present at start of section
            label_match = self.label_pattern.search(section_content[:200])
            if label_match:
                section_node.label = label_match.group(1)
            
            # Add leaf nodes (sentences, formulas, figures)
            section_node.children = self._create_leaf_nodes(section_content, source_file)
            
            parent.children.append(section_node)
            stack.append((section_node, level))
        
        return root
    
    def parse_with_file_markers(self, merged_content: str) -> HierarchyNode:
        """Parse merged content that has file markers"""
        # Split by file markers
        file_pattern = re.compile(r'%%% FILE: (.+?) %%%\n')
        parts = file_pattern.split(merged_content)
        
        # Create root
        root = HierarchyNode(
            node_type=NodeType.DOCUMENT,
            title="Document"
        )
        
        # Process each file's content
        current_file = ""
        for i, part in enumerate(parts):
            if i % 2 == 1:  # File path
                current_file = part
            elif part.strip():  # Content
                file_root = self.parse(part, current_file)
                # Merge children into main root
                root.children.extend(file_root.children)
        
        return root


In [46]:
class BibTeXExtractor:
    """Extract and convert bibliography entries from .bbl files and .bib files"""
    
    # Pattern to match \bibitem entries in .bbl files
    BIBITEM_PATTERN = re.compile(
        r'\\bibitem\[([^\]]*)\]\{([^}]+)\}\s*(.*?)(?=\\bibitem|\Z|\\end\{thebibliography\})',
        re.DOTALL
    )
    
    # Alternative pattern without optional argument
    BIBITEM_SIMPLE_PATTERN = re.compile(
        r'\\bibitem\{([^}]+)\}\s*(.*?)(?=\\bibitem|\Z|\\end\{thebibliography\})',
        re.DOTALL
    )
    
    # Pattern for BibTeX entries in .bib files  
    BIBTEX_ENTRY_PATTERN = re.compile(
        r'@(\w+)\{([^,]+),\s*(.*?)\n\}',
        re.DOTALL
    )
    
    # Pattern for BibTeX fields
    BIBTEX_FIELD_PATTERN = re.compile(
        r'(\w+)\s*=\s*\{([^{}]*(?:\{[^{}]*\}[^{}]*)*)\}',
        re.DOTALL
    )
    
    def __init__(self):
        self.entries: Dict[str, BibEntry] = {}
        
    def parse_bib_file(self, content: str) -> Dict[str, BibEntry]:
        """Parse a .bib file and extract entries"""
        entries = {}
        
        for match in self.BIBTEX_ENTRY_PATTERN.finditer(content):
            entry_type = match.group(1).lower()
            key = match.group(2).strip()
            fields_str = match.group(3)
            
            # Parse fields
            fields = {}
            for field_match in self.BIBTEX_FIELD_PATTERN.finditer(fields_str):
                field_name = field_match.group(1).lower()
                field_value = field_match.group(2).strip()
                fields[field_name] = field_value
            
            entries[key] = BibEntry(key=key, entry_type=entry_type, fields=fields)
        
        return entries
    
    def parse_bbl_file(self, content: str) -> Dict[str, BibEntry]:
        """Parse a .bbl file and convert bibitem entries to BibTeX format"""
        entries = {}
        
        # Try pattern with optional citation label
        for match in self.BIBITEM_PATTERN.finditer(content):
            cite_label = match.group(1)
            key = match.group(2).strip()
            entry_content = match.group(3).strip()
            
            entry = self._parse_bibitem_content(key, entry_content, cite_label)
            if entry:
                entries[key] = entry
        
        # If no entries found, try simple pattern
        if not entries:
            for match in self.BIBITEM_SIMPLE_PATTERN.finditer(content):
                key = match.group(1).strip()
                entry_content = match.group(2).strip()
                
                entry = self._parse_bibitem_content(key, entry_content)
                if entry:
                    entries[key] = entry
        
        return entries
    
    def _parse_bibitem_content(self, key: str, content: str, cite_label: str = "") -> Optional[BibEntry]:
        """Parse the content of a bibitem and extract bibliographic fields"""
        fields = {}
        
        # Clean up LaTeX commands
        content = self._clean_latex(content)
        
        # Try to extract structured information
        # Extract authors (usually at the beginning)
        author_match = re.match(r'^([^.]+?)\.', content)
        if author_match:
            authors = author_match.group(1).strip()
            authors = self._normalize_authors(authors)
            fields['author'] = authors
            content = content[author_match.end():].strip()
        
        # Extract title (usually in \newblock or after authors)
        title_match = re.search(r'\\newblock\s*\{?([^.]+)\}?\.', content)
        if title_match:
            fields['title'] = title_match.group(1).strip()
        else:
            # Try to find title as first sentence after author
            title_match = re.match(r'^([^.]+)\.', content)
            if title_match:
                fields['title'] = title_match.group(1).strip()
        
        # Extract year
        year_match = re.search(r'\b(19|20)\d{2}\b', content)
        if year_match:
            fields['year'] = year_match.group(0)
        
        # Extract journal/booktitle (often in \emph{})
        journal_match = re.search(r'\\emph\{([^}]+)\}', content)
        if journal_match:
            venue = journal_match.group(1).strip()
            # Guess if it's a journal or conference
            if any(kw in venue.lower() for kw in ['proc', 'conf', 'workshop', 'symposium']):
                fields['booktitle'] = venue
            else:
                fields['journal'] = venue
        
        # Extract URL
        url_match = re.search(r'\\url\{([^}]+)\}', content)
        if url_match:
            fields['url'] = url_match.group(1)
        
        # Extract DOI
        doi_match = re.search(r'doi:\s*([^\s,}]+)', content, re.IGNORECASE)
        if doi_match:
            fields['doi'] = doi_match.group(1)
        
        # Extract pages
        pages_match = re.search(r'pages?\s*[:\s]*(\d+(?:\s*[-–]\s*\d+)?)', content, re.IGNORECASE)
        if pages_match:
            fields['pages'] = pages_match.group(1).replace('–', '--')
        
        # Extract volume
        volume_match = re.search(r'vol(?:ume)?\.?\s*(\d+)', content, re.IGNORECASE)
        if volume_match:
            fields['volume'] = volume_match.group(1)
        
        # Determine entry type
        entry_type = self._guess_entry_type(fields, content)
        
        # Store original content as note if parsing was incomplete
        if len(fields) < 3:
            fields['note'] = content[:500]  # Truncate long notes
        
        return BibEntry(key=key, entry_type=entry_type, fields=fields)
    
    def _clean_latex(self, content: str) -> str:
        """Remove common LaTeX formatting commands"""
        # Remove \newblock
        content = re.sub(r'\\newblock\s*', '', content)
        # Remove common formatting
        content = re.sub(r'\\textit\{([^}]*)\}', r'\1', content)
        content = re.sub(r'\\textbf\{([^}]*)\}', r'\1', content)
        content = re.sub(r'\\texttt\{([^}]*)\}', r'\1', content)
        content = re.sub(r'\\emph\{([^}]*)\}', r'\1', content)
        # Remove escaped characters
        content = content.replace('\\&', '&')
        content = content.replace('\\~', '~')
        content = content.replace('\\{', '{')
        content = content.replace('\\}', '}')
        # Normalize whitespace
        content = re.sub(r'\s+', ' ', content)
        return content.strip()
    
    def _normalize_authors(self, authors: str) -> str:
        """Normalize author names to BibTeX format"""
        # Replace 'and' with ' and '
        authors = re.sub(r'\s+and\s+', ' and ', authors)
        # Replace commas between authors (but not in names)
        # Handle "et al."
        authors = re.sub(r',?\s*et\s*al\.?', ' and others', authors)
        return authors.strip()
    
    def _guess_entry_type(self, fields: Dict[str, str], content: str) -> str:
        """Guess the BibTeX entry type based on available fields"""
        content_lower = content.lower()
        
        if 'booktitle' in fields or 'proceedings' in content_lower or 'conference' in content_lower:
            return 'inproceedings'
        elif 'journal' in fields:
            return 'article'
        elif 'url' in fields and ('howpublished' in content_lower or 'accessed' in content_lower):
            return 'misc'
        elif 'publisher' in content_lower or 'press' in content_lower:
            return 'book'
        elif 'thesis' in content_lower:
            if 'phd' in content_lower or 'doctoral' in content_lower:
                return 'phdthesis'
            elif 'master' in content_lower:
                return 'mastersthesis'
        elif 'arxiv' in content_lower:
            return 'article'
        
        return 'misc'
    
    def load_from_directory(self, base_dir: str) -> Dict[str, BibEntry]:
        """Load all bibliography entries from .bib and .bbl files in directory"""
        base_path = Path(base_dir)
        
        # Load .bib files
        for bib_file in base_path.rglob('*.bib'):
            try:
                with open(bib_file, 'r', encoding='utf-8', errors='replace') as f:
                    content = f.read()
                entries = self.parse_bib_file(content)
                self.entries.update(entries)
                print(f"Loaded {len(entries)} entries from {bib_file.name}")
            except Exception as e:
                print(f"Error loading {bib_file}: {e}")
        
        # Load .bbl files
        for bbl_file in base_path.rglob('*.bbl'):
            try:
                with open(bbl_file, 'r', encoding='utf-8', errors='replace') as f:
                    content = f.read()
                entries = self.parse_bbl_file(content)
                # Only add entries not already present from .bib files
                for key, entry in entries.items():
                    if key not in self.entries:
                        self.entries[key] = entry
                print(f"Loaded {len(entries)} entries from {bbl_file.name}")
            except Exception as e:
                print(f"Error loading {bbl_file}: {e}")
        
        return self.entries
    
    def to_bibtex(self) -> str:
        """Export all entries to BibTeX format"""
        return '\n\n'.join(entry.to_bibtex() for entry in self.entries.values())
    
    def get_entries(self) -> Dict[str, BibEntry]:
        """Get all loaded entries"""
        return self.entries


In [47]:
class Deduplicator:
    """Handles deduplication of references and content"""
    
    def __init__(self):
        self.content_hashes: Dict[str, str] = {}  # hash -> unique_id
        self.reference_hashes: Dict[str, str] = {}  # content_hash -> key
        
    # ==================== Reference Deduplication ====================
    
    def deduplicate_references(self, entries: Dict[str, BibEntry]) -> Dict[str, BibEntry]:
        """
        Deduplicate bibliography entries.
        Entries with different keys but same content are merged.
        """
        deduplicated = {}
        hash_to_key: Dict[str, str] = {}
        key_mappings: Dict[str, str] = {}  # old_key -> canonical_key
        
        for key, entry in entries.items():
            content_hash = entry.content_hash()
            
            if content_hash in hash_to_key:
                # Duplicate found - merge fields
                canonical_key = hash_to_key[content_hash]
                canonical_entry = deduplicated[canonical_key]
                
                # Merge fields (prefer non-empty values)
                for field, value in entry.fields.items():
                    if field not in canonical_entry.fields or not canonical_entry.fields[field]:
                        canonical_entry.fields[field] = value
                
                key_mappings[key] = canonical_key
                print(f"Merged duplicate: {key} -> {canonical_key}")
            else:
                # New unique entry
                hash_to_key[content_hash] = key
                deduplicated[key] = entry
                key_mappings[key] = key
        
        self.reference_hashes = hash_to_key
        return deduplicated
    
    def find_similar_references(self, entries: Dict[str, BibEntry], 
                                 threshold: float = 0.8) -> List[Tuple[str, str, float]]:
        """
        Find potentially duplicate references using fuzzy matching.
        Returns list of (key1, key2, similarity_score)
        """
        similar_pairs = []
        keys = list(entries.keys())
        
        for i, key1 in enumerate(keys):
            for key2 in keys[i+1:]:
                entry1 = entries[key1]
                entry2 = entries[key2]
                
                similarity = self._compute_entry_similarity(entry1, entry2)
                if similarity >= threshold:
                    similar_pairs.append((key1, key2, similarity))
        
        return sorted(similar_pairs, key=lambda x: -x[2])
    
    def _compute_entry_similarity(self, entry1: BibEntry, entry2: BibEntry) -> float:
        """Compute similarity between two bibliography entries"""
        scores = []
        
        # Compare titles
        if 'title' in entry1.fields and 'title' in entry2.fields:
            title_sim = self._string_similarity(
                entry1.fields['title'].lower(),
                entry2.fields['title'].lower()
            )
            scores.append(title_sim * 2)  # Weight title higher
        
        # Compare authors
        if 'author' in entry1.fields and 'author' in entry2.fields:
            author_sim = self._string_similarity(
                entry1.fields['author'].lower(),
                entry2.fields['author'].lower()
            )
            scores.append(author_sim * 1.5)
        
        # Compare year
        if 'year' in entry1.fields and 'year' in entry2.fields:
            year_sim = 1.0 if entry1.fields['year'] == entry2.fields['year'] else 0.0
            scores.append(year_sim)
        
        if not scores:
            return 0.0
        
        return sum(scores) / (2 + 1.5 + 1)  # Normalized by weights
    
    def _string_similarity(self, s1: str, s2: str) -> float:
        """Compute similarity between two strings using Jaccard similarity on words"""
        words1 = set(s1.split())
        words2 = set(s2.split())
        
        if not words1 or not words2:
            return 0.0
        
        intersection = len(words1 & words2)
        union = len(words1 | words2)
        
        return intersection / union if union > 0 else 0.0
    
    # ==================== Content Deduplication ====================
    
    def deduplicate_content(self, root: HierarchyNode) -> HierarchyNode:
        """
        Deduplicate content in the hierarchy tree.
        Identical content elements are given the same unique_id.
        """
        self.content_hashes.clear()
        return self._deduplicate_node(root)
    
    def _deduplicate_node(self, node: HierarchyNode) -> HierarchyNode:
        """Recursively deduplicate a node and its children"""
        # Process children first
        deduplicated_children = []
        for child in node.children:
            deduplicated_child = self._deduplicate_node(child)
            deduplicated_children.append(deduplicated_child)
        
        node.children = deduplicated_children
        
        # For leaf nodes, check for duplicates
        if node.node_type in LEAF_TYPES and node.content:
            content_hash = node.content_hash
            
            if content_hash in self.content_hashes:
                # Use existing unique_id for duplicate content
                node.unique_id = self.content_hashes[content_hash]
            else:
                # Store this as the canonical version
                self.content_hashes[content_hash] = node.unique_id
        
        return node
    
    def get_duplicate_stats(self, root: HierarchyNode) -> Dict[str, Any]:
        """Get statistics about duplicates in the hierarchy"""
        stats = {
            'total_nodes': 0,
            'unique_contents': 0,
            'duplicate_contents': 0,
            'by_type': defaultdict(lambda: {'total': 0, 'unique': 0})
        }
        
        seen_hashes = set()
        
        def count_node(node: HierarchyNode):
            stats['total_nodes'] += 1
            stats['by_type'][node.node_type.value]['total'] += 1
            
            if node.node_type in LEAF_TYPES and node.content_hash:
                if node.content_hash not in seen_hashes:
                    seen_hashes.add(node.content_hash)
                    stats['unique_contents'] += 1
                    stats['by_type'][node.node_type.value]['unique'] += 1
                else:
                    stats['duplicate_contents'] += 1
            
            for child in node.children:
                count_node(child)
        
        count_node(root)
        return stats
    
    def find_duplicate_content(self, root: HierarchyNode) -> Dict[str, List[HierarchyNode]]:
        """Find all duplicate content items grouped by content hash"""
        hash_to_nodes: Dict[str, List[HierarchyNode]] = defaultdict(list)
        
        def collect_node(node: HierarchyNode):
            if node.node_type in LEAF_TYPES and node.content_hash:
                hash_to_nodes[node.content_hash].append(node)
            for child in node.children:
                collect_node(child)
        
        collect_node(root)
        
        # Return only groups with duplicates
        return {h: nodes for h, nodes in hash_to_nodes.items() if len(nodes) > 1}


In [48]:
class LaTeXParser:
    """
    Main LaTeX parser that integrates all components:
    - Multi-file gathering
    - Hierarchy construction
    - Reference extraction
    - Deduplication
    """
    
    def __init__(self, base_dir: str):
        self.base_dir = Path(base_dir)
        self.gatherer = LaTeXFileGatherer(base_dir)
        self.cleaner = LaTeXCleaner()
        self.hierarchy_parser = HierarchyParser(self.cleaner)
        self.bib_extractor = BibTeXExtractor()
        self.deduplicator = Deduplicator()
        
        # Results
        self.hierarchy: Optional[HierarchyNode] = None
        self.references: Dict[str, BibEntry] = {}
        self.file_contents: Dict[str, str] = {}
        
    def parse(self, main_file: str = "main.tex") -> Dict[str, Any]:
        """
        Parse the LaTeX document starting from the main file.
        Returns a dictionary with hierarchy, references, and statistics.
        """
        print(f"Parsing LaTeX document from: {self.base_dir / main_file}")
        print("=" * 60)
        
        # Step 1: Multi-file gathering
        print("\n[1] Gathering files...")
        self.file_contents = self.gatherer.gather_files(main_file)
        print(f"    Found {len(self.file_contents)} files in compilation path")
        
        # Report unused files
        unused = self.gatherer.get_unused_files()
        if unused:
            print(f"    Unused files ({len(unused)}):")
            for f in list(unused)[:5]:
                print(f"      - {Path(f).name}")
            if len(unused) > 5:
                print(f"      ... and {len(unused) - 5} more")
        
        # Step 2: Build hierarchy
        print("\n[2] Building hierarchy...")
        merged_content = self.gatherer.get_merged_content()
        self.hierarchy = self.hierarchy_parser.parse_with_file_markers(merged_content)
        
        hierarchy_stats = self._count_hierarchy_nodes(self.hierarchy)
        print(f"    Built hierarchy with {hierarchy_stats['total']} nodes:")
        for node_type, count in hierarchy_stats['by_type'].items():
            print(f"      - {node_type}: {count}")
        
        # Step 3: Extract references
        print("\n[3] Extracting references...")
        self.references = self.bib_extractor.load_from_directory(str(self.base_dir))
        print(f"    Found {len(self.references)} bibliography entries")
        
        # Step 4: Deduplicate references
        print("\n[4] Deduplicating references...")
        original_count = len(self.references)
        self.references = self.deduplicator.deduplicate_references(self.references)
        dedup_count = original_count - len(self.references)
        print(f"    Removed {dedup_count} duplicate references")
        
        # Find similar references (potential duplicates with different keys)
        similar = self.deduplicator.find_similar_references(self.references, threshold=0.7)
        if similar:
            print(f"    Found {len(similar)} pairs of similar references:")
            for key1, key2, sim in similar[:3]:
                print(f"      - '{key1}' ~ '{key2}' (similarity: {sim:.2%})")
        
        # Step 5: Deduplicate content
        print("\n[5] Deduplicating content...")
        self.hierarchy = self.deduplicator.deduplicate_content(self.hierarchy)
        dup_stats = self.deduplicator.get_duplicate_stats(self.hierarchy)
        print(f"    Unique content items: {dup_stats['unique_contents']}")
        print(f"    Duplicate content items: {dup_stats['duplicate_contents']}")
        
        print("\n" + "=" * 60)
        print("Parsing complete!")
        
        return {
            'hierarchy': self.hierarchy,
            'references': self.references,
            'file_contents': self.file_contents,
            'stats': {
                'files': len(self.file_contents),
                'unused_files': len(unused),
                'hierarchy_nodes': hierarchy_stats,
                'references': len(self.references),
                'duplicate_refs_removed': dedup_count,
                'content_dedup': dup_stats
            }
        }
    
    def _count_hierarchy_nodes(self, node: HierarchyNode) -> Dict[str, Any]:
        """Count nodes in hierarchy by type"""
        stats = {'total': 0, 'by_type': defaultdict(int)}
        
        def count(n: HierarchyNode):
            stats['total'] += 1
            stats['by_type'][n.node_type.value] += 1
            for child in n.children:
                count(child)
        
        count(node)
        return stats
    
    def to_json(self, include_content: bool = True) -> str:
        """Export parsed document to JSON"""
        if not self.hierarchy:
            return "{}"
        
        data = {
            'hierarchy': self.hierarchy.to_dict(),
            'references': {
                key: {
                    'type': entry.entry_type,
                    'key': entry.key,
                    'fields': entry.fields
                }
                for key, entry in self.references.items()
            },
            'metadata': {
                'base_dir': str(self.base_dir),
                'files_count': len(self.file_contents),
                'references_count': len(self.references)
            }
        }
        
        return json.dumps(data, indent=2, ensure_ascii=False)
    
    def export_bibtex(self) -> str:
        """Export references to BibTeX format"""
        return self.bib_extractor.to_bibtex()
    
    def print_hierarchy(self, node: HierarchyNode = None, indent: int = 0):
        """Print hierarchy tree for debugging"""
        if node is None:
            node = self.hierarchy
        
        if node is None:
            print("No hierarchy parsed yet")
            return
        
        prefix = "  " * indent
        type_str = node.node_type.value.upper()
        title_str = f": {node.title[:50]}..." if node.title and len(node.title) > 50 else f": {node.title}" if node.title else ""
        content_preview = ""
        
        if node.node_type in LEAF_TYPES and node.content:
            content_preview = f" [{node.content[:40]}...]"
        
        print(f"{prefix}[{type_str}]{title_str}{content_preview}")
        
        for child in node.children:
            self.print_hierarchy(child, indent + 1)
    
    def get_all_sentences(self) -> List[str]:
        """Extract all sentences from the hierarchy"""
        sentences = []
        
        def extract(node: HierarchyNode):
            if node.node_type == NodeType.SENTENCE:
                sentences.append(node.content)
            for child in node.children:
                extract(child)
        
        if self.hierarchy:
            extract(self.hierarchy)
        
        return sentences
    
    def get_all_formulas(self) -> List[str]:
        """Extract all block formulas from the hierarchy"""
        formulas = []
        
        def extract(node: HierarchyNode):
            if node.node_type == NodeType.BLOCK_FORMULA:
                formulas.append(node.content)
            for child in node.children:
                extract(child)
        
        if self.hierarchy:
            extract(self.hierarchy)
        
        return formulas
    
    def get_all_figures(self) -> List[Dict[str, str]]:
        """Extract all figures from the hierarchy"""
        figures = []
        
        def extract(node: HierarchyNode):
            if node.node_type == NodeType.FIGURE:
                figures.append({
                    'label': node.label,
                    'caption': node.title,
                    'content': node.content
                })
            for child in node.children:
                extract(child)
        
        if self.hierarchy:
            extract(self.hierarchy)
        
        return figures


In [49]:
# Print the hierarchy structure (limited depth for readability)
print("Document Hierarchy (top levels):")
print("=" * 60)

def print_limited_hierarchy(node, max_depth=3, current_depth=0):
    """Print hierarchy with depth limit"""
    if current_depth > max_depth:
        return
    
    prefix = "  " * current_depth
    type_str = node.node_type.value.upper()
    title = node.title[:50] + "..." if node.title and len(node.title) > 50 else node.title
    
    leaf_count = sum(1 for c in node.children if c.node_type in LEAF_TYPES)
    child_count = len(node.children) - leaf_count
    
    suffix = ""
    if leaf_count > 0:
        suffix = f" ({leaf_count} leaf elements)"
    
    print(f"{prefix}[{type_str}] {title}{suffix}")
    
    for child in node.children:
        if child.node_type not in LEAF_TYPES:
            print_limited_hierarchy(child, max_depth, current_depth + 1)

print_limited_hierarchy(parser.hierarchy)


Document Hierarchy (top levels):
[DOCUMENT] Document
  [CHAPTER] Introduction
  [CHAPTER] \nlgsec
    [SENTENCE] 
  [CHAPTER] \nlusec
    [SENTENCE] 
  [CHAPTER] \autolabsec
    [SENTENCE] 
  [CHAPTER] \aidosec
    [SENTENCE] 
  [CHAPTER] \sharcsec
  [CHAPTER] \moocsec
  [CHAPTER] Conclusion
  [FIGURE] Temp figure: {#1
  [SENTENCE] 
  [SENTENCE] 
  [SENTENCE] 
  [SENTENCE] 
  [ACKNOWLEDGMENTS] Acknowledgments
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
  [CHAPTER] Abstract
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [SENTENCE] 
    [S

In [50]:
class MilestoneExporter:
    """
    Export parsed LaTeX document to the milestone-required format.
    
    Format for <yymm-id>.json:
    {
        "elements": {
            "id-string-1": "Cleaned latex content of element 1",
            ...
        },
        "hierarchy": {
            "1": {  // Version 1
                "child-id": "parent-id",
                ...
            }
        }
    }
    """
    
    def __init__(self):
        self.elements: Dict[str, str] = {}  # unique_id -> cleaned content
        self.hierarchy: Dict[str, Dict[str, str]] = {}  # version -> {child_id: parent_id}
        
    def export_document(self, root: HierarchyNode, version: str = "1") -> Dict[str, Any]:
        """
        Export a parsed document to the milestone format.
        
        Args:
            root: The root HierarchyNode of the parsed document
            version: Version identifier (e.g., "1", "2", etc.)
            
        Returns:
            Dictionary in the required format
        """
        self.elements.clear()
        self.hierarchy.clear()
        
        # Process the hierarchy tree
        self._process_node(root, parent_id=None, version=version)
        
        return {
            "elements": self.elements,
            "hierarchy": self.hierarchy
        }
    
    def _process_node(self, node: HierarchyNode, parent_id: Optional[str], version: str):
        """Recursively process nodes to extract elements and build hierarchy"""
        
        # Initialize version hierarchy if not exists
        if version not in self.hierarchy:
            self.hierarchy[version] = {}
        
        current_id = node.unique_id
        
        # For leaf nodes, store the content in elements
        if node.node_type in LEAF_TYPES:
            if node.content:
                # Only store if not already present (deduplication)
                if current_id not in self.elements:
                    self.elements[current_id] = node.content
                
                # Add to hierarchy (child -> parent relationship)
                if parent_id:
                    self.hierarchy[version][current_id] = parent_id
        else:
            # For non-leaf nodes, also track in hierarchy but don't store content
            # Store section/chapter titles as content for context
            if node.title:
                content = f"[{node.node_type.value.upper()}] {node.title}"
                if current_id not in self.elements:
                    self.elements[current_id] = content
            
            # Add to hierarchy
            if parent_id:
                self.hierarchy[version][current_id] = parent_id
        
        # Process children
        for child in node.children:
            self._process_node(child, current_id, version)
    
    def merge_versions(self, versions_data: List[Tuple[HierarchyNode, str]]) -> Dict[str, Any]:
        """
        Merge multiple versions into a single export.
        Elements are deduplicated across versions.
        
        Args:
            versions_data: List of (root_node, version_id) tuples
            
        Returns:
            Dictionary with merged elements and per-version hierarchies
        """
        self.elements.clear()
        self.hierarchy.clear()
        
        for root, version in versions_data:
            self._process_node(root, parent_id=None, version=version)
        
        return {
            "elements": self.elements,
            "hierarchy": self.hierarchy
        }
    
    def save_to_file(self, data: Dict[str, Any], output_path: str):
        """Save exported data to JSON file"""
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print(f"Saved to {output_path}")
    
    @staticmethod
    def get_arxiv_id_from_path(base_dir: str) -> str:
        """
        Extract arXiv ID from directory name.
        Expected format: yymm.nnnnn or yymm-nnnnn
        """
        dir_name = Path(base_dir).name
        # Try to match arXiv ID pattern
        match = re.match(r'(\d{4})[.\-](\d{4,5})', dir_name)
        if match:
            return f"{match.group(1)}.{match.group(2)}"
        return dir_name


In [ ]:
class MainFileDetector:
    """Automatically detect the main LaTeX file in a directory."""
    
    # Common main file names
    MAIN_FILE_CANDIDATES = [
        'main.tex', 'paper.tex', 'article.tex', 'manuscript.tex',
        'thesis.tex', 'document.tex', 'root.tex'
    ]
    
    # Document class pattern to identify main files
    DOCUMENT_CLASS_PATTERN = re.compile(r'\\documentclass', re.IGNORECASE)
    BEGIN_DOCUMENT_PATTERN = re.compile(r'\\begin\{document\}', re.IGNORECASE)
    AUTHOR_DOCUMENT_PATTERN = re.compile(r'\\author', re.IGNORECASE)
    
    @classmethod
    def find_main_file(cls, tex_dir: str) -> Optional[str]:
        """
        Find the main LaTeX file in a directory.
        
        Strategy:
        1. Look for common main file names
        2. Look for files with \\documentclass that also have \\begin{document}
        3. If only one .tex file at root level, use that
        """
        tex_path = Path(tex_dir)
        
        if not tex_path.exists():
            return None
        
        # Strategy 1: Check common names
        for candidate in cls.MAIN_FILE_CANDIDATES:
            candidate_path = tex_path / candidate
            if candidate_path.exists():
                return candidate
        
        # Strategy 2: Find files with documentclass AND begin{document}
        root_tex_files = list(tex_path.glob('*.tex'))
        main_candidates = []
        
        for tex_file in root_tex_files:
            try:
                with open(tex_file, 'r', encoding='utf-8', errors='replace') as f:
                    content = f.read()
                
                has_docclass = cls.DOCUMENT_CLASS_PATTERN.search(content)
                has_begin_doc = cls.BEGIN_DOCUMENT_PATTERN.search(content)
                has_author = cls.AUTHOR_DOCUMENT_PATTERN.search(content)
                
                if has_docclass and has_begin_doc and has_author:
                    main_candidates.append(tex_file.name)
            except Exception:
                continue
        
        if len(main_candidates) == 1:
            return main_candidates[0]
        elif len(main_candidates) > 1:
            # Prefer shorter filenames (less likely to be appendix, etc.)
            return min(main_candidates, key=len)
        
        # Strategy 3: If only one tex file at root level
        if len(root_tex_files) == 1:
            return root_tex_files[0].name
        
        # Strategy 4: Look for any tex file with documentclass
        for tex_file in root_tex_files:
            try:
                with open(tex_file, 'r', encoding='utf-8', errors='replace') as f:
                    content = f.read()
                if cls.DOCUMENT_CLASS_PATTERN.search(content):
                    return tex_file.name
            except Exception:
                continue
        
        return None


class MultiVersionProcessor:
    """Process multiple versions of the same paper."""
    
    def __init__(self, paper_dir: str):
        self.paper_dir = Path(paper_dir)
        self.arxiv_id = self.paper_dir.name
        # self.tex_dir = self.paper_dir / 'tex'
        
        # Check if tex/ subdirectory exists, otherwise use paper_dir directly
        tex_dir_candidate = self.paper_dir / 'tex'
        self.tex_dir = tex_dir_candidate if tex_dir_candidate.exists() else self.paper_dir
        
        self.versions: Dict[str, Path] = {}
        self.results: Dict[str, Any] = {}
        
    def discover_versions(self) -> List[str]:
        """Find all version directories (e.g., 2305-14596v1, 2305-14596v2)."""
        self.versions.clear()
        
        if not self.tex_dir.exists():
            return []
        
        # Look for version folders
        version_pattern = re.compile(rf'{re.escape(self.arxiv_id)}v(\d+)', re.IGNORECASE)
        
        for item in self.tex_dir.iterdir():
            if item.is_dir():
                match = version_pattern.match(item.name)
                if match:
                    version_num = match.group(1)
                    self.versions[version_num] = item
        
        # Sort versions numerically
        return sorted(self.versions.keys(), key=int)
    
    def parse_version(self, version: str) -> Optional[Dict[str, Any]]:
        """Parse a specific version of the paper."""
        if version not in self.versions:
            return None
        
        version_dir = self.versions[version]
        main_file = MainFileDetector.find_main_file(str(version_dir))
        
        if not main_file:
            print(f"  Warning: Could not find main file in {version_dir}")
            return None
        
        try:
            parser = LaTeXParser(str(version_dir))
            result = parser.parse(main_file)
            return {
                'parser': parser,
                'result': result,
                'main_file': main_file
            }
        except Exception as e:
            print(f"  Error parsing version {version}: {e}")
            return None
    
    def process_all_versions(self) -> Dict[str, Any]:
        """Process all versions and return combined results."""
        versions = self.discover_versions()
        
        if not versions:
            print(f"  No versions found for {self.arxiv_id}")
            return {}
        
        print(f"  Found {len(versions)} version(s): {versions}")
        
        for version in versions:
            print(f"\n  Processing version {version}...")
            result = self.parse_version(version)
            if result:
                self.results[version] = result
        
        return self.results
    
    def export_combined(self, output_dir: str) -> Optional[Path]:
        """
        Export combined results for all versions.
        Elements are deduplicated across versions.
        Each version has its own hierarchy.
        """
        if not self.results:
            return None
        
        exporter = MilestoneExporter()
        combined_elements = {}
        combined_hierarchy = {}
        
        for version, data in self.results.items():
            parser = data['parser']
            if parser.hierarchy:
                # Export this version
                version_data = exporter.export_document(parser.hierarchy, version=version)
                
                # Merge elements (deduplicated by ID)
                combined_elements.update(version_data['elements'])
                
                # Add version hierarchy
                combined_hierarchy.update(version_data['hierarchy'])
        
        # Create output structure
        output_data = {
            'elements': combined_elements,
            'hierarchy': combined_hierarchy
        }
        
        # Create output directory
        out_path = Path(output_dir) / self.arxiv_id
        out_path.mkdir(parents=True, exist_ok=True)
        
        # Save main JSON file
        main_json = out_path / f"{self.arxiv_id}.json"
        with open(main_json, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)
        
        # Save metadata from source if exists
        metadata_src = self.paper_dir / 'metadata.json'
        if metadata_src.exists():
            import shutil
            shutil.copy(metadata_src, out_path / 'metadata.json')
        
        # Save references from source if exists
        references_src = self.paper_dir / 'references.json'
        if references_src.exists():
            import shutil
            shutil.copy(references_src, out_path / 'references.json')
        
        # Export BibTeX from all versions
        all_bibtex = {}
        for version, data in self.results.items():
            parser = data['parser']
            for key, entry in parser.references.items():
                if key not in all_bibtex:
                    all_bibtex[key] = {
                        'type': entry.entry_type,
                        'fields': entry.fields
                    }
        
        if all_bibtex:
            bibtex_json = out_path / 'extracted_bibtex.json'
            with open(bibtex_json, 'w', encoding='utf-8') as f:
                json.dump(all_bibtex, f, indent=2, ensure_ascii=False)
        
        return out_path

In [ ]:
class BatchProcessor:
    """
    Process all papers in a student folder.
    
    Expected folder structure:
    student_folder/
        2305-14596/
            metadata.json
            references.json
            tex/
                2305-14596v1/
                    main.tex
                    ...
                2305-14596v2/
                    main.tex
                    ...
        2305-14597/
            ...
    """
    
    def __init__(self, student_folder: str, output_folder: str = None):
        self.student_folder = Path(student_folder)
        self.student_id = self.student_folder.name
        self.output_folder = Path(output_folder) if output_folder else self.student_folder.parent / f"{self.student_id}_output"
        
        self.papers: List[Path] = []
        self.results: Dict[str, Any] = {}
        self.stats = {
            'total_papers': 0,
            'processed': 0,
            'failed': 0,
            'skipped': 0,
            'total_elements': 0,
            'total_versions': 0
        }
    
    def discover_papers(self) -> List[str]:
        """Find all paper directories in the student folder."""
        self.papers.clear()
        
        # Look for directories matching arXiv ID pattern
        arxiv_pattern = re.compile(r'\d{4}-\d{4,5}')
        
        for item in sorted(self.student_folder.iterdir()):
            if item.is_dir() and arxiv_pattern.match(item.name):
                # Accept folders with tex/ subdirectory OR version folders directly
                tex_dir = item / 'tex'
                # If tex_dir.exists(): # If it has a tex/ subdirectory
                has_tex = tex_dir.exists()
                has_versions = any(
                    (item / d.name).is_dir() and re.match(rf'{re.escape(item.name)}v\d+', d.name, re.IGNORECASE)
                    for d in item.iterdir() if d.is_dir()
                )
                if has_tex or has_versions:
                    self.papers.append(item)
        
        self.stats['total_papers'] = len(self.papers)
        return [p.name for p in self.papers]
    
    def process_paper(self, paper_dir: Path) -> Optional[Dict[str, Any]]:
        """Process a single paper with all its versions."""
        processor = MultiVersionProcessor(str(paper_dir))
        results = processor.process_all_versions()
        
        if results:
            output_path = processor.export_combined(str(self.output_folder))
            
            # Calculate stats
            total_elements = 0
            for version_data in results.values():
                parser = version_data.get('parser')
                if parser and parser.hierarchy:
                    def count_nodes(node):
                        count = 1
                        for child in node.children:
                            count += count_nodes(child)
                        return count
                    total_elements += count_nodes(parser.hierarchy)
            
            return {
                'arxiv_id': paper_dir.name,
                'versions_processed': len(results),
                'output_path': output_path,
                'total_elements': total_elements
            }
        
        return None
    
    def process_all(self, limit: int = None, verbose: bool = True) -> Dict[str, Any]:
        """
        Process all papers in the folder.
        
        Args:
            limit: Maximum number of papers to process (for testing)
            verbose: Whether to print progress
        """
        papers = self.discover_papers()
        
        if not papers:
            print(f"No papers found in {self.student_folder}")
            return self.stats
        
        print(f"{'='*60}")
        print(f"BATCH PROCESSING: {self.student_id}")
        print(f"{'='*60}")
        print(f"Found {len(papers)} papers to process")
        print(f"Output folder: {self.output_folder}")
        print(f"{'='*60}\n")
        
        # Create output folder
        self.output_folder.mkdir(parents=True, exist_ok=True)
        
        papers_to_process = self.papers[:limit] if limit else self.papers
        
        for i, paper_dir in enumerate(papers_to_process):
            arxiv_id = paper_dir.name
            
            if verbose:
                print(f"\n[{i+1}/{len(papers_to_process)}] Processing {arxiv_id}...")
                print("-" * 40)
            
            try:
                result = self.process_paper(paper_dir)
                
                if result:
                    self.results[arxiv_id] = result
                    self.stats['processed'] += 1
                    self.stats['total_versions'] += result['versions_processed']
                    self.stats['total_elements'] += result['total_elements']
                    
                    if verbose:
                        print(f"  ✓ Successfully processed {result['versions_processed']} version(s)")
                        print(f"    Elements: {result['total_elements']}")
                else:
                    self.stats['skipped'] += 1
                    if verbose:
                        print(f"  ⚠ Skipped (no versions found or parse failed)")
                        
            except Exception as e:
                self.stats['failed'] += 1
                if verbose:
                    print(f"  ✗ Failed: {str(e)}")
                import traceback
                traceback.print_exc()
        
        # Print summary
        print(f"\n{'='*60}")
        print("BATCH PROCESSING COMPLETE")
        print(f"{'='*60}")
        print(f"📊 Statistics:")
        print(f"   - Total papers found: {self.stats['total_papers']}")
        print(f"   - Successfully processed: {self.stats['processed']}")
        print(f"   - Skipped: {self.stats['skipped']}")
        print(f"   - Failed: {self.stats['failed']}")
        print(f"   - Total versions processed: {self.stats['total_versions']}")
        print(f"   - Total elements extracted: {self.stats['total_elements']}")
        print(f"\n📁 Output folder: {self.output_folder}")
        
        # Save processing summary
        summary_file = self.output_folder / 'processing_summary.json'
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump({
                'student_id': self.student_id,
                'stats': self.stats,
                'papers': {
                    arxiv_id: {
                        'versions_processed': r['versions_processed'],
                        'total_elements': r['total_elements']
                    }
                    for arxiv_id, r in self.results.items()
                }
            }, f, indent=2, ensure_ascii=False)
        
        print(f"📝 Summary saved to: {summary_file}")
        
        return self.stats


In [53]:
# Test with a few papers first
STUDENT_FOLDER = "23127238"
OUTPUT_FOLDER = "23127238_output"

# Initialize batch processor
batch_processor = BatchProcessor(STUDENT_FOLDER, OUTPUT_FOLDER)

# Discover all papers
papers = batch_processor.discover_papers()
print(f"Found {len(papers)} papers in {STUDENT_FOLDER}")

stats = batch_processor.process_all(limit=5, verbose=True)

Found 14326 papers in 23127238
BATCH PROCESSING: 23127238
Found 14326 papers to process
Output folder: 23127238_output


[1/5] Processing 2304-14607...
----------------------------------------
  Found 1 version(s): ['1']

  Processing version 1...
  ⚠ Skipped (no versions found or parse failed)

[2/5] Processing 2304-14608...
----------------------------------------
  Found 2 version(s): ['1', '2']

  Processing version 1...
Parsing LaTeX document from: 23127238\2304-14608\tex\2304-14608v1\light_source_2023.tex

[1] Gathering files...
    Found 1 files in compilation path

[2] Building hierarchy...
    Built hierarchy with 236 nodes:
      - document: 1
      - abstract: 1
      - sentence: 195
      - section: 7
      - figure: 16
      - subsection: 10
      - table: 1
      - subsubsection: 4
      - acknowledgments: 1

[3] Extracting references...
Loaded 18 entries from lightsourcebib.bib
    Found 18 bibliography entries

[4] Deduplicating references...
Merged duplicate: IceCube:2